# Less Naive representation pipeline (pennylane)

Main steps:

* input: image of the sky (frequency domain)
* apply 2D Fourier transform --> visibilities (Fourier domain)
* encode classical visibilities data into qbits (non-conventional domain)
* measure qbits (back to conventional domain)
* apply 2D Inverse Fourier transform --> original image?

In [8]:
import numpy as np
import math
import sys

import pennylane as qml
from pennylane import numpy as pnp
#from pennylane.templates.embeddings import AmplitudeEmbedding

## Generating an artificial image of the sky

In [14]:
#image of the sky filled with single precision complex floats
#pixels are set to low complex random values (image background) 
#few pixels are set to larger complex random values in a specified ellipse area (image source/subject)

n = 4
image = np.zeros((n, n), dtype='complex64')
image.real = np.random.rand(n , n) / 100
image.imag = np.random.rand(n , n) / 100

h, w = image.shape
mask = circular_mask(h, w, radius=h/2)
sky_image = image.copy()
sky_image[~mask] = complex(np.random.rand() * 100, np.random.rand() * 100)
print(sky_image)

[[2.7510828e+01+2.8234703e+01j 2.7510828e+01+2.8234703e+01j
  3.6175237e-03+6.4585963e-03j 2.7510828e+01+2.8234703e+01j]
 [2.7510828e+01+2.8234703e+01j 4.3533933e-03+4.4468590e-03j
  6.9055124e-03+3.3520055e-03j 8.2349451e-03+6.3901028e-04j]
 [4.6377857e-03+6.7809406e-03j 7.5805178e-03+4.4731712e-03j
  3.4381095e-03+1.3682940e-03j 6.6085169e-03+6.2355092e-03j]
 [2.7510828e+01+2.8234703e+01j 8.3669601e-03+7.8799566e-03j
  1.0267316e-03+8.3748708e-03j 1.2169689e-03+6.1689648e-03j]]


## Applying 2D Fourier transform (visibilities)

In [15]:
visibilities = np.fft.fft2(sky_image).astype('complex64')
print(visibilities)

[[137.61012 +141.22969j   82.525894 +84.687096j  27.494093 +28.231194j
   82.51838  +84.69558j ]
 [ 82.49985  +84.68282j   27.523829 +28.227585j -27.497158 -28.22856j
   27.49824  +28.229836j]
 [ 27.506605 +28.229158j -27.512594 -28.221725j -82.52074  -84.6928j
  -27.498032 -28.226318j]
 [ 82.527824 +84.70059j   27.491718 +28.220022j -27.505037 -28.22281j
   27.510258 +28.213884j]]


### Sanity check

In [16]:
img = np.fft.ifft2(visibilities).astype('complex64')
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()
 
print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[2.7510828e+01+2.8234703e+01j 2.7510828e+01+2.8234703e+01j
  3.6168098e-03+6.4579248e-03j 2.7510828e+01+2.8234703e+01j]
 [2.7510828e+01+2.8234703e+01j 4.3535233e-03+4.4466257e-03j
  6.9049597e-03+3.3512115e-03j 8.2349777e-03+6.3836575e-04j]
 [4.6381950e-03+6.7812204e-03j 7.5801611e-03+4.4734478e-03j
  3.4377575e-03+1.3681650e-03j 6.6078901e-03+6.2355995e-03j]
 [2.7510828e+01+2.8234703e+01j 8.3670616e-03+7.8800917e-03j
  1.0260344e-03+8.3749294e-03j 1.2164116e-03+6.1689615e-03j]]

Reals MSE: 1.5322406e-13
Imaginaries MSE: 1.0952636e-13


## Classical data encoding/decoding

In [36]:
# Amplitude embedding encodes a normalized 2^n-dimensional feature vector into the state of n qbits (uses log2(n) qbits for n classical data)
n = visibilities.shape[0]*visibilities.shape[1]
data = visibilities.flatten()

#normalization to prepare a qstate with measurement probabilites summing up to 1 (SUM (amplitudes²) = 1)
norm = qml.math.sum(qml.math.abs(data) ** 2)
normalized_data = data / qml.math.sqrt(norm)

wires = range(int(math.log2(n))) # set the number of qbits (no padding needed if outputs an integer=integer.0)
amp_dev = qml.device('default.qubit', wires)

@qml.qnode(amp_dev)
def amp_encoding(data):
    qml.AmplitudeEmbedding(data, wires) # normalize = True
    return qml.state() #qml.expval(qml.PauliZ(wires=wires))

results = amp_encoding(normalized_data).astype('complex64')
print(results)

print()

results = results*qml.math.sqrt(norm) # denormalization of the measurements outcomes
results = np.array(results).reshape(sky_image.shape[0] , sky_image.shape[1])
print(results)

[ 0.39027768+0.40054318j  0.23405266+0.24018206j  0.07797632+0.08006682j
  0.23403133+0.24020612j  0.23397878+0.24016994j  0.07806065+0.08005659j
 -0.07798501-0.08005935j  0.07798808+0.08006296j  0.0780118 +0.08006105j
 -0.07802879-0.08003996j -0.23403803-0.24019825j -0.07798749-0.08005299j
  0.23405813+0.24022034j  0.07796958+0.08003514j -0.07800736-0.08004304j
  0.07802217+0.08001773j]

[[137.61012 +141.22969j   82.525894 +84.687096j  27.494093 +28.231194j
   82.51838  +84.69558j ]
 [ 82.49985  +84.68282j   27.523829 +28.227587j -27.497158 -28.228561j
   27.498241 +28.229836j]
 [ 27.506605 +28.22916j  -27.512594 -28.221725j -82.52074  -84.69281j
  -27.498034 -28.22632j ]
 [ 82.527824 +84.70059j   27.491718 +28.220022j -27.505037 -28.22281j
   27.51026  +28.213886j]]


## Applying 2D Inverse Fourier transform (+ fidelity test)

In [43]:
img = np.fft.ifft2(results).astype('complex64')
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()

print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[2.7510828e+01+2.8234703e+01j 2.7510828e+01+2.8234703e+01j
  3.6166906e-03+6.4573288e-03j 2.7510828e+01+2.8234703e+01j]
 [2.7510828e+01+2.8234703e+01j 4.3535233e-03+4.4456720e-03j
  6.9049597e-03+3.3514500e-03j 8.2347393e-03+6.3812733e-04j]
 [4.6378374e-03+6.7806244e-03j 7.5800419e-03+4.4742823e-03j
  3.4381151e-03+1.3682842e-03j 6.6080093e-03+6.2357187e-03j]
 [2.7510828e+01+2.8234703e+01j 8.3673000e-03+7.8796148e-03j
  1.0257959e-03+8.3751678e-03j 1.2164116e-03+6.1682463e-03j]]

Reals MSE: 1.7794387e-13
Imaginaries MSE: 3.8772224e-13


## Utils

### Quantum

### Classical

In [9]:
#creates a circular mask over a 2D array
def circular_mask(h, w, center=None, radius=None):
    if center is None: #image center
        center = (int(w/2), int(h/2))
    if radius is None: #smallest distance between center and image bounderies
        radius = min(center[0], center[1], w-center[0], h-center[1])
        
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    mask = dist_from_center <= radius
    
    return mask